# Sequence To SQL

Welcome to our project in the Advanced Natural Language Processing course

We try to build it with the data provided in https://github.com/salesforce/WikiSQL

Remove this: https://towardsdatascience.com/text-to-sql-learning-to-query-tables-with-natural-language-7d714e60a70d?gi=6b6c7e91e298

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
import torch

if torch.cuda.is_available():
    device = torch.cuda.current_device()
    print('Current device:', torch.cuda.get_device_name(device))
else:
    print('Failed to find GPU. Will use CPU.')
    device = 'cpu'

# Data collection and Review
Clone the data from the WikiSQL git repository and install them.

Take a look inside the data

In [ ]:
import json

def read_json_data_from_file(file: str):
  ret_data = []
  with open(file) as json_file:
    while True:
      # Get next line from file
      line = json_file.readline()
      if not line:
        break

      data = json.loads(line)
      ret_data.append(data)
  return ret_data

def convert_to_id_dict(data, id_key: str):
  ret_dict = {}
  for element in data:
    if id_key in element:
      ret_dict[element[id_key]] = element
    else:
      print(f'Element {element} doenst contain key {id_key}')
  return ret_dict

Lets see if we succesfully serialized the data into objects.

In [ ]:
data_folder = "../data"

dev_req_data = read_json_data_from_file(f'{data_folder}/dev.jsonl')
dev_table_data = read_json_data_from_file(f'{data_folder}/dev.tables.jsonl')
 
print(f'We have {len(dev_req_data)} dev data with {len(dev_table_data)} tables.')
print(f'An example Request: ')
print(json.dumps(dev_req_data[0], indent=2))

### The fields represent the following:

* `phase`: the phase in which the dataset was collected. We collected WikiSQL in two phases.
* `question`: the natural language question written by the worker.
* `table_id`: the ID of the table to which this question is addressed.
sql: the SQL query corresponding to the question. This has the following *subfields:
  * `sel`: the numerical index of the column that is being selected. You can find the actual column from the table.
  * `agg`: the numerical index of the aggregation operator that is being used. You can find the actual operator from Query.agg_ops in lib/query.py.
  * `conds`: a list of triplets (column_index, operator_index, condition) where:
    * `column_index`: the numerical index of the condition column that is being used. You can find the actual column from the table.
    * `operator_index`: the numerical index of the condition operator that is being used. You can find the actual operator from Query.cond_ops in lib/query.py.
    * `condition`: the comparison value for the condition, in either string or float type.


## An example Table

In [ ]:
print(json.dumps(dev_table_data[0], indent=2))

## Preprocess

The data is stored with indices but we need the actual column names so saturate the requests with the data

In [ ]:
# Transform the data into a dictonary index by the id
dev_table_data_dict = convert_to_id_dict(dev_table_data, 'id')

In [ ]:
# Get the preliminary data
# Maybe we want the other idexes also

In [ ]:
def get_table_column(data_list, tables_dict):
  ret_list = []
  for element in data_list:
    current_table = tables_dict[element['table_id']]
    columns = current_table['header']
    # Replace the index
    element['columns'] = columns
    element['types'] = current_table['types']
    element['sql']['sel'] = columns[element['sql']['sel']]

    if 'page_title' in current_table:
        element['table_name'] = current_table['page_title']
    elif 'section_title' in current_table:
        element['table_name'] = current_table['section_title']
    elif 'caption' in current_table:
        element['table_name'] = current_table['caption']
    elif 'name' in current_table:
        element['table_name'] = current_table['name']

    # For the where conditions
    for cond in element['sql']['conds']:
      cond[0] = columns[cond[0]]
    ret_list.append(element)
  return ret_list


In [ ]:
dev_req_data = read_json_data_from_file(f'{data_folder}/dev.jsonl')
dev_table_data = read_json_data_from_file(f'{data_folder}/dev.tables.jsonl')

dev_prep_req_data = get_table_column(dev_req_data, dev_table_data_dict)


print(f'Filed in with the Columns: ')
print(json.dumps(dev_prep_req_data[-2], indent=2))


In [ ]:
pad_max_length = 50

def get_question_answers(request, tokenizer):
    input_list = []

    table_name = request['table_name'] #should be name not id
    space_token = ' '
    columns = request['columns']
    req_question = request['question'] # might need to be tokenized
    max_len = 0
    for i, col in enumerate(columns):
        col_type = request['types'][i] # infere type somehow
        column_representation = col_type + space_token + table_name + space_token + col
        embedding = tokenizer.encode_plus(
            column_representation,
            req_question,
            add_special_tokens=True,
        )
        if max_len < len(embedding['input_ids']):
            max_len = len(embedding['input_ids'])

    for i, col in enumerate(columns):
        col_type = request['types'][i] # infere type somehow
        column_representation = col_type + space_token + table_name + space_token + col
        embedding = tokenizer.encode_plus(
            column_representation,
            req_question,
            add_special_tokens=True,
            max_length=max_len,
            padding=True,
            return_overflowing_tokens=True,
        )
        input_list.append(embedding)
    return input_list

In [ ]:
pad_max_length = 50
def get_question_answers(request, tokenizer):
    input_list = []

    table_name = request['table_name'] #should be name not id
    space_token = ' '
    columns = request['columns']
    req_question = request['question'] # might need to be tokenized
    for i, col in enumerate(columns):
        col_type = request['types'][i] # infere type somehow
        column_representation = col_type + space_token + table_name + space_token + col
        embedding = tokenizer.encode_plus(
            column_representation,
            req_question,
            add_special_tokens=True,
            max_length=pad_max_length,
            pad_to_max_length=True,
            return_overflowing_tokens=True,
        )
        input_list.append(embedding)
    return input_list

# Build the model

In this section, we will look into **contextual embeddings**. 

For this we use [**pretrained BERT**](https://www.aclweb.org/anthology/N19-1423.pdf) provided via [HuggingFace](https://huggingface.co/).

Let's first install the HuggingFace python package:

In [ ]:
# Dp imports here
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

In [ ]:
print(dev_prep_req_data[0])
for tokens in get_question_answers(dev_prep_req_data[0], tokenizer):
    print(tokens)
    print(tokenizer.convert_ids_to_tokens(tokens['input_ids']))

In [ ]:
# Generall model configuration
PRE_TRAINED_MODEL_NAME = 'bert-base-cased'
sep_token = tokenizer.sep_token
cls_token = tokenizer.cls_token

In [ ]:
# reference is hydranet https://arxiv.org/pdf/2008.04759.pdf

In [ ]:
token_lens = []

for data in dev_prep_req_data:
    c_split_data = tokenizer.tokenize(data['question'])
    for line in c_split_data:
        tokens = tokenizer.encode(line, max_length=512)
        token_lens.append(len(tokens))

In [ ]:
sns.distplot(token_lens)
plt.xlim([0, 50]);
plt.xlabel('Token count');

In [ ]:
from transformers import BertModel

model = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)

In [ ]:
req_embeddings = get_question_answers(dev_prep_req_data[0], tokenizer)
print(req_embeddings)
input_ids = [req_embedding['input_ids'] for req_embedding in req_embeddings]
token_type_ids = [req_embedding['token_type_ids'] for req_embedding in req_embeddings]
attention_mask = [req_embedding['attention_mask'] for req_embedding in req_embeddings]

In [ ]:
print(input_ids)
print(token_type_ids)
print(attention_mask)

In [ ]:
outputs = model(
    input_ids=torch.tensor(input_ids), # The tokens representing our input text.
    attention_mask=torch.tensor(attention_mask),
    token_type_ids=torch.tensor(token_type_ids)
) # The segment IDs to differentiate question from answer_text
outputs

In [ ]:
print(f'last hidden state  : {outputs.last_hidden_state.shape}')
print(f'pooled output layer: {outputs.pooler_output.shape}')

In [ ]:
from torch import nn
import torch.nn.functional as F

linear = nn.Linear(model.config.hidden_size, 6)

linear(outputs.pooler_output)
F.softmax(linear(outputs.pooler_output), dim=1)

In [ ]:
answer_start = torch.argmax(outputs.start_logits)
answer_end = torch.argmax(outputs.end_logits)

text = tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end])

print(f'{text}: {answer_start} {answer_end}')